### Imports

In [42]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re 

In [3]:
li_titles = []
li_descriptions = []
li_bedrooms = []
li_prices = []

### Put all together

In [4]:
for i in range(0, 300, 20):
    website = 'https://www.airbnb.com/s/San-Francisco--CA--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&query=San%20Francisco%2C%20CA%2C%20United%20States&place_id=ChIJIQBpAG2ahYAR_6128GcTUEo&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click&price_filter_input_type=0&federated_search_session_id=212d076c-c587-41f6-86f3-4d46f7f86ca8&pagination_search=true&items_offset=' + str(i) + '&section_offset=2'
    response = requests.get(website)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    results = soup.findAll('div',  {'class' : "g1tup9az"})
    titles = [t.find('div', {'class' : "t1jojoys dir dir-ltr"}).get_text() for t in results]
    descriptions = [d.find('span', {'class' : "tjbvqj3 dir dir-ltr"}).get_text() for d in results]
    bedrooms = [b.find('span', {'class' : "dir dir-ltr"}).get_text() for b in results]
    prices = [p.find('span', {'class' : "a8jt5op dir dir-ltr"}).get_text() for p in results]
    
    for k in range(len(results)):
        li_titles.append(titles[k])
        li_descriptions.append([descriptions[k]])                 
        li_bedrooms.append(bedrooms[k])                  
        li_prices.append(prices[k])                   

### Information about Dataframe

In [5]:
real_estate_all_pages = pd.DataFrame({'Title': li_titles, 
                                     'Description' : li_descriptions, 
                                     'Bedrooms' : li_bedrooms,
                                     'Price' : li_prices})

In [7]:
real_estate_all_pages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        300 non-null    object
 1   Description  300 non-null    object
 2   Bedrooms     300 non-null    object
 3   Price        300 non-null    object
dtypes: object(4)
memory usage: 9.5+ KB


In [8]:
real_estate_all_pages.head()

,Title,Description,Bedrooms,Price
0,Apartment in Nob Hill,"[1000 Pine st, San Francisco 94109]",2 beds,"$169 per night, originally $204"
1,Hotel room in Marina District,[Quiet Marina District Garden Oasis Free Parking],1 queen bed,$205 per night
2,Apartment in Pacific Heights,[The Addison by Kasa | Traditional Queen Studio],Sep 2 – 9,"$244 per night, originally $272"
3,Guest suite in Mission District,[Mission Private 1BR/BA Garden Suite Separate ...,1 queen bed,$292 per night
4,Guest suite in West Portal Park,[Cozy live/work suite in safe area by Metro su...,1 bed,"$99 per night, originally $129"


In [9]:
real_estate_all_pages.tail()

,Title,Description,Bedrooms,Price
295,Condo in San Francisco,[Top Area/Lux Suite/Garden nr Fillmore & Union...,1 king bed,$484 per night
296,Private room in San Francisco,[Luxury Harry Potter Guestroom],2 beds,"$159 per night, originally $179"
297,Home in San Francisco,[Modern 1 Bedroom Garden Suite - Lovely & Bright],2 beds,$245 per night
298,Hotel room in San Francisco,[Residential Quad Family Suite],3 beds,$175 per night
299,Guest suite in San Francisco,[One guest private room next to Golden Gate Park],1 queen bed,"$134 per night, originally $149"


### Data Cleaning

In [10]:
real_estate_all_pages['Description'] = real_estate_all_pages['Description'].apply(lambda x: 
                                                                                  str(x).replace('[',''))
real_estate_all_pages['Description'] = real_estate_all_pages['Description'].apply(lambda x: 
                                                                                  str(x).replace(']',''))
real_estate_all_pages['Description'] = real_estate_all_pages['Description'].apply(lambda x: 
                                                                                  str(x).replace("'",''))
real_estate_all_pages['Description'] = real_estate_all_pages['Description'].apply(lambda x: 
                                                                                  str(x).replace('*',''))

In [11]:
real_estate_all_pages.head()

,Title,Description,Bedrooms,Price
0,Apartment in Nob Hill,"1000 Pine st, San Francisco 94109",2 beds,"$169 per night, originally $204"
1,Hotel room in Marina District,Quiet Marina District Garden Oasis Free Parking,1 queen bed,$205 per night
2,Apartment in Pacific Heights,The Addison by Kasa | Traditional Queen Studio,Sep 2 – 9,"$244 per night, originally $272"
3,Guest suite in Mission District,Mission Private 1BR/BA Garden Suite Separate E...,1 queen bed,$292 per night
4,Guest suite in West Portal Park,Cozy live/work suite in safe area by Metro subway,1 bed,"$99 per night, originally $129"


In [54]:
se_price = pd.Series(real_estate_all_pages['Price'])
se_price_new = []
for items in se_price.iteritems():
    index, text = (items)
    result = re.findall('[0-9]+', text)[0]
    se_price_new.append(f'${result} per night')

In [57]:
real_estate_all_pages['Price'] = pd.Series(se_price_new)
real_estate_all_pages

,Title,Description,Bedrooms,Price
0,Apartment in Nob Hill,"1000 Pine st, San Francisco 94109",2 beds,$169 per night
1,Hotel room in Marina District,Quiet Marina District Garden Oasis Free Parking,1 queen bed,$205 per night
2,Apartment in Pacific Heights,The Addison by Kasa | Traditional Queen Studio,Sep 2 – 9,$244 per night
3,Guest suite in Mission District,Mission Private 1BR/BA Garden Suite Separate E...,1 queen bed,$292 per night
4,Guest suite in West Portal Park,Cozy live/work suite in safe area by Metro subway,1 bed,$99 per night
...,...,...,...,...
295,Condo in San Francisco,Top Area/Lux Suite/Garden nr Fillmore & Union ...,1 king bed,$484 per night
296,Private room in San Francisco,Luxury Harry Potter Guestroom,2 beds,$159 per night
297,Home in San Francisco,Modern 1 Bedroom Garden Suite - Lovely & Bright,2 beds,$245 per night
298,Hotel room in San Francisco,Residential Quad Family Suite,3 beds,$175 per night


### Save in Excel

In [58]:
real_estate_all_pages.to_excel('real_estate_all_pages.xlsx', index = False) 